In [31]:
from __future__ import absolute_import, division, print_function
import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.8.0'

In [32]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

In [33]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy
                 )
    return model

In [34]:
model = create_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [35]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10)

Epoch 1/10
32/32 [==============================] - 1s 5ms/step - loss: 1.1456
Epoch 2/10
32/32 [==============================] - 0s 5ms/step - loss: 0.4170
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2921
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2262
Epoch 5/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1527
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1159
Epoch 7/10
32/32 [==============================] - 0s 5ms/step - loss: 0.0932
Epoch 8/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0659
Epoch 9/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0489
Epoch 10/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0369


In [36]:
!ls {checkpoint_dir}

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [52]:
model = create_model()
acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 2ms/step - loss: 2.3958
Untrained model, accuracy: 239.58%


In [53]:
model.load_weights(checkpoint_path)
acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./cp.ckpt

In [40]:
model.save("model.h5")

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open('h5.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ryguy\AppData\Local\Temp\tmp9evt3sp3\assets


INFO:tensorflow:Assets written to: C:\Users\ryguy\AppData\Local\Temp\tmp9evt3sp3\assets


In [48]:
tflite_model_name = 'sine_model'
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

1629712

In [54]:
def hex_to_c_array(hex_data, var_name):
    
    c_str = ''
    
    # create header guard
    c_str += '#ifndef' + var_name.upper() + '_H\n'
    c_str += '#define' + var_name.upper() + '_H\n\n'
    
    # add array length at top of file
    c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'
    
    # declare c variable
    c_str += 'unsigned char' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) : 
        
        # construct string from hex
        hex_str = format(val, '#04x')
        
        # add formatting so each line stays within 80 chars
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) %12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)
        
    # add closing brace
    c_str += '\n' + format(''.join(hex_array)) + '\n};\n\n'
    
    # close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'
    
    return c_str

In [56]:
c_model_name = 'c_sine'
with open(c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [58]:
model.save(tflite_model_name + '.h5')